### Make ternary plots starting from saved .pkl files

In [4]:
from functions import *

#### Load relevant data

In [71]:
EDS_path = r"Z:\P110143-phosphosulfides-Andrea\Data\Analysis\guidal\mittma_pickles"
names = ["04", "06", "07", "08" ] # which samples to load for the plot
author = "mittma"

dfs={}
for name in names:
    df = []
    
    filename = f"{author}_00{name}_EDS.pkl"
    EDS_pkl = os.path.join(EDS_path, filename)
    
    with open(EDS_pkl, 'rb') as f:
       df= pickle.load(f)
    dfs[f"{author}_00{name}"] = df


In [72]:
EDS_df = pd.DataFrame()
for key in dfs:
    print(key)
    df = dfs[key]
    EDS_df = combine_data((EDS_df, df))

mittma_0004
mittma_0006
mittma_0007
mittma_0008


In [172]:
def ternary_plot(df, el1, el2, el3, datatype, title, savepath = None): 
    "make a ternary plot of the data in df, with el1, el2, el3 as the corners, and colorscale based on datatype"

    A= f'Layer 1 {el1} Atomic %'
    B= f'Layer 1 {el2} Atomic %'
    C= f'Layer 1 {el3} Atomic %'

    A_percent = df.xs(A, level='Data type', axis=1).values.flatten()
    B_percent = df.xs(B, level='Data type', axis=1).values.flatten()
    C_percent = df.xs(C, level='Data type', axis=1).values.flatten()
    intensity = df.xs(datatype, level='Data type', axis=1).values.flatten()
    coordinates = MI_to_grid(df).values # df.columns.get_level_values(0)[::7].values.flatten() also works, but gives a lot of decimals

    fig = go.Figure()
    fig.add_trace(go.Scatterternary({
        'mode': 'markers',
        'a': A_percent,   # el1 percentages
        'b': B_percent,   # el2 percentages
        'c': C_percent,   # el3 percentages
        'marker': {
            'symbol': 100 ,
            'size': 8,
            'color': intensity,  # Use intensity for marker color
            'colorscale': 'Turbo',  # Choose a colorscale
            'colorbar': {'title': datatype},  # Add a colorbar
            'line': {'width': 2}
        },
        'text': coordinates,
        'hovertemplate': f'{el1}: %{{a:.1f}}%<br>{el2}: %{{b:.1f}}%<br>{el3}: %{{c:.1f}}%<br>{datatype}: %{{marker.color:.1f}}<br>Coordinates:%{{text:str}}',  # Custom hover text format
        #'name': f'{datatype}: %{{text:str}}'
        'showlegend': False
    }))

    # Update layout
    fig.update_layout({
        'ternary': {
            'sum': 100,
            'aaxis': {'title': f'{el1} %', 'min': 0, 'linewidth': 2, 'ticks': 'outside'},
            'baxis': {'title': f'{el2} %', 'min': 0, 'linewidth': 2, 'ticks': 'outside'},
            'caxis': {'title': f'{el3} %', 'min': 0, 'linewidth': 2, 'ticks': 'outside'}
        },
        'title': title},
        width=800,
        height=600, 
    )       
    if savepath:
        if savepath.endswith(".png"):
            fig.write_image(savepath, scale=2)
        if savepath.endswith(".html"):
            fig.write_html(savepath)

    # Show the plot
    fig.show()
    

In [174]:

datatype = "Sample ID"
T=200
#title = f"{author}" +" samples at 200 "
title = f"Cu-P-S samples at {T} \u00b0C"
#savepath =os.path.join(EDS_path, f'{author}_{T}C'+"_ternary.png")

ternary_plot(EDS_df, "Cu", "P", "S", datatype, title)
